In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/machine learning/Speech-Emotion-Recognition/Paper/

/content/drive/MyDrive/machine learning/Speech-Emotion-Recognition


In [3]:
! git checkout dev

M	Paper/paper.ipynb
Already on 'dev'
Your branch is up to date with 'origin/dev'.


In [5]:
! pip install tensorflow-io==0.23.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports and Deterministic Behaviour

In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import re
import numpy as np
import matplotlib.pyplot as plt

import data_operations as data_ops

import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D,  MaxPooling1D, BatchNormalization, GlobalMaxPooling1D

import pandas as pd

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [7]:
DATA_DIR = "./data/SAVEE"

TRAIN_SIZE, VAL_SIZE, TEST_SIZE = (300, 100, 80)

SAMPLE_RATE_HZ = 44100

# Load audio and Get Label functions

In [8]:
def get_label(file_path):
  parts = re.sub('.+\_|[0-9]+.wav', '', file_path)
  return parts

In [9]:
filenames = [f'{DATA_DIR}/{p}' for p in os.listdir(DATA_DIR)]
np.random.shuffle(filenames)
labels = pd.get_dummies(list(map(get_label, filenames))).to_numpy()

In [10]:
train_files, train_labels = (filenames[:TRAIN_SIZE], labels[:TRAIN_SIZE])
val_files, val_labels = (filenames[TRAIN_SIZE: TRAIN_SIZE + VAL_SIZE], labels[TRAIN_SIZE: TRAIN_SIZE + VAL_SIZE])
test_files, test_labels = (filenames[-TEST_SIZE:], labels[-TEST_SIZE:])

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))
(train_files[0], train_labels[0])

Training set size 300
Validation set size 100
Test set size 80


('./data/SAVEE/DC_n21.wav', array([0, 0, 0, 0, 1, 0, 0], dtype=uint8))

In [11]:
operations = [
    data_ops.ReadFile(),
    data_ops.DecodeWav(),
    data_ops.Squeeze(),
    data_ops.Crop(SAMPLE_RATE_HZ * 8),
    data_ops.ZeroPad(SAMPLE_RATE_HZ * 8),
    data_ops.CastToFloat(),
    data_ops.Reshape((SAMPLE_RATE_HZ * 8, 1))
]

train_ds = tfio.audio.AudioIODataset.from_tensor_slices((train_files, train_labels))
val_ds = tfio.audio.AudioIODataset.from_tensor_slices((val_files, val_labels))

In [12]:
for o in operations:
    train_ds = train_ds.map(o, num_parallel_calls=tf.data.AUTOTUNE)

In [13]:
for o in operations:
    val_ds = val_ds.map(o, num_parallel_calls=tf.data.AUTOTUNE)

In [14]:
# rows = 3
# cols = 3
# n = rows * cols
# fig, axes = plt.subplots(rows, cols, figsize=(12, 15))

# for i, (audio, label) in enumerate(train_ds.take(n)):
#   r = i // cols
#   c = i % cols
#   ax = axes[r][c]
#   ax.plot(audio.numpy())
#   print(audio.numpy().shape)
#   label = label.numpy().decode('utf-8')
#   ax.set_title(label)

# plt.show()

## CNN Architecture
 * 6x 1D + Batch norm + max pooling
 * 1x 1D + Batch norm + global max pooling
 * 2x dense layer

In [15]:
first_filter_size =  (32, 21)

filters = [64, 128, 256, 512, 1024]
sizes = [19, 17, 15, 13, 11]

middle_filters_size = list(zip(filters, sizes))
last_filter_size =  (1024, 9)

activation= 'relu'
pool_size = 2

In [16]:
model = Sequential()

# first layer
# input shape (None, n) = variable-length sequences of n-dimensional vectors
model.add(Conv1D(first_filter_size[0], first_filter_size[1], activation = activation, input_shape=(SAMPLE_RATE_HZ * 8, 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size = pool_size))

# middle layers
for (filter_size, kernel_size) in middle_filters_size:
    model.add(Conv1D(filter_size, kernel_size, activation = activation))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = pool_size))

# last layer
model.add(Conv1D(last_filter_size[0], last_filter_size[1], activation = activation))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D())

# model.add(Flatten())
model.add(Dense(128, activation=activation))
model.add(Dense(7, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 352780, 32)        704       
                                                                 
 batch_normalization (BatchN  (None, 352780, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 176390, 32)       0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 176372, 64)        38976     
                                                                 
 batch_normalization_1 (Batc  (None, 176372, 64)       256       
 hNormalization)                                                 
                                                        

In [17]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
train_ds = train_ds.batch(8)
val_ds = val_ds.batch(8)

EPOCHS = 400

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./tmp/checkpoint')

In [ ]:
# model.fit(train_files['padded'].map(lambda x : tf.convert_to_tensor(x.numpy())), train_files['label'], epochs=10, validation_data=(val_files['padded'].map(lambda x : tf.convert_to_tensor(x.numpy())), val_files['label']))
model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[checkpoint_callback])

Epoch 1/400
38/38 [==============================] - 166s 4s/step - loss: 6.1456 - accuracy: 0.2333 - val_loss: 165.2055 - val_accuracy: 0.1400
Epoch 2/400
38/38 [==============================] - 131s 3s/step - loss: 2.5658 - accuracy: 0.2833 - val_loss: 11.6813 - val_accuracy: 0.1500
Epoch 3/400
38/38 [==============================] - 130s 3s/step - loss: 2.0619 - accuracy: 0.3100 - val_loss: 2.2331 - val_accuracy: 0.3200
Epoch 4/400
38/38 [==============================] - 130s 3s/step - loss: 1.8620 - accuracy: 0.3367 - val_loss: 1.9033 - val_accuracy: 0.2600
Epoch 5/400
38/38 [==============================] - 130s 3s/step - loss: 1.8205 - accuracy: 0.3300 - val_loss: 1.8211 - val_accuracy: 0.2300
Epoch 6/400
38/38 [==============================] - 130s 3s/step - loss: 1.7034 - accuracy: 0.3367 - val_loss: 1.7868 - val_accuracy: 0.2600
Epoch 7/400
38/38 [==============================] - 129s 3s/step - loss: 1.6415 - accuracy: 0.3567 - val_loss: 1.6969 - val_accuracy: 0.2600
Epo